# Aggregations, Window Functions, and UDFs 

## Simple aggregations

### How to do it...

In [ ]:
import polars as pl

In [ ]:
df = pl.read_csv('../data/constoso_sales.csv', try_parse_dates=True)

In [ ]:
df.head()

In [ ]:
from polars import selectors as cs
(
    df
    .select(cs.numeric())
    .sum()
)

In [ ]:
s = df.select('Quantity').to_series()
s.sum()

In [ ]:
df.select(pl.col('Quantity').sum())

In [ ]:
df.select(
    pl.col('Customer Name').first().alias('Cust Name First'),
    pl.col('Customer Name').last().alias('Cust Name Last')
)

In [ ]:
df.select(cs.numeric()).describe()

### There is more...

In [ ]:
df.select(
    (pl.col('Quantity') >= 4).sum()
)

In [ ]:
df.select(
    pl.col('Quantity').filter(pl.col('Store Name')=='Online store').sum()
)

## Using group by aggregations

### How to do it...

In [ ]:
import polars as pl

In [ ]:
df = pl.read_csv('../data/constoso_sales.csv', try_parse_dates=True)
df.head()

In [ ]:
df.group_by('Brand')

In [ ]:
(
    df
    .group_by('Brand')
    .agg(pl.col('Quantity').sum().alias('Sum of Quantity'))
    .head()
)

In [ ]:
(
    df
    .group_by('Brand')
    .agg(
        pl.col('Unit Price').sum().alias('Sum of Quantity'),
        pl.col('Unit Price').mean().alias('Average Unit Price'),
    )
    .sort('Average Unit Price', descending=True)
    .head()
)

In [ ]:
(
    df
    .group_by('Brand')
    .agg(
        pl.col('Unit Price').mean().round(2).alias('Average Unit Price'),
        (pl.col('Unit Price').sum() / pl.count()).round(2).alias('Average Unit Price 2'),
        pl.col('Customer Name').first(),
        pl.col('Category').last()
    )
    .sort('Average Unit Price', descending=True)
    .sort('Brand')
    .head()
)

In [ ]:
(
    pl.scan_csv('../data/constoso_sales.csv', try_parse_dates=True)
    .group_by('Brand')
    .agg(
        pl.col('Unit Price').mean().round(2).alias('Average Unit Price'),
        (pl.col('Unit Price').sum() / pl.count()).round(2).alias('Average Unit Price 2'),
        pl.col('Customer Name').first(),
        pl.col('Category').last()
    )
    .sort('Average Unit Price', descending=True)
    .sort('Brand')
    .collect()
    .head()
)

In [ ]:
pl.Config.set_fmt_str_lengths = 50
print(df.select('Brand').unique().head(10))

In [ ]:
df.select('Brand').unique().head(10)

In [ ]:
import os
os.environ['POLARS_FMT_STR_LEN'] = str(50)

df.select('Brand').unique().head(10)

### There is more...

In [ ]:
for name, data in df.group_by('Brand'):
    print(name, type(data))

In [ ]:
(
    df
    .group_by('Brand')
    .agg(pl.col('Quantity'))
    .head()
)

In [ ]:
(
    df
    .group_by('Brand', maintain_order=True)
    .agg(pl.col('Quantity'))
    .head()
)

In [ ]:
(
    df
    .group_by(
        pl.col('Brand'), 
        'Customer Country',
        pl.col('Order Date').dt.year().alias('Order Year')
        )
    .agg(pl.col('Unit Price').mean())
    .head()
)

## Aggregating values across multiple columns

### How to do it...

In [ ]:
import polars as pl

In [163]:
df = pl.read_csv('../data/pokemon.csv')
df.head()

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false
3,"""VenusaurMega Venusaur""","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false


In [164]:
(
    df
    .select('HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed')
    .sum(axis=1).alias('Total 2')
    .head(5)
)

Total 2
i64
318
405
525
625
309


In [165]:
(
    df
    .with_columns(
        pl.sum_horizontal('HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed').alias('Total 2')
    )
    .head(5)
)

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total 2
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool,i64
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false,318
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false,405
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false,525
3,"""VenusaurMega Venusaur""","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false,625
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false,309


In [167]:
(
    df
    .with_columns(
        pl.concat_list('HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed').list.sum().alias('Total 2')
    )
    .head(5)
)

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total 2
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool,i64
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false,318
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false,405
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false,525
3,"""VenusaurMega Venusaur""","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false,625
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false,309


In [ ]:
cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
(
    df
    .with_columns(
        pl.reduce(
            function=lambda acc, col: acc + col, 
            exprs=pl.col(cols)
        )
        .alias('Total 2')
    )
    .head()
)

In [ ]:
(
    df
    .with_columns(
        pl.fold(
            acc=pl.lit(100), 
            function=lambda acc, col: acc + col, 
            exprs=pl.col(cols)
        )
        .alias('Total 2')
    )
    .head()
)

### There is more...

In [ ]:
(
    df
    .filter(
        pl.fold(
            acc=pl.lit(True), 
            function=lambda acc, col: acc & col, 
            exprs=pl.col(cols) > 80
        )
    )
    .head()
)

In [ ]:
(
    df
    .filter(
        pl.all_horizontal(pl.col(cols) > 80)
    )
    .head()
)

In [ ]:
str_cols = ['Name', 'Type 1', 'Type 2']
str_combined = pl.fold(acc=pl.lit(''), function=lambda acc, col: acc + col, exprs=str_cols).alias('Str Combined')
str_cols.append(str_combined)
df.select(str_cols).head()

In [ ]:
str_cols = ['Name', 'Type 1', 'Type 2']
df.select(pl.concat_str(str_cols)).head()

## Computing over groups with window functions

### How to do it...

In [1]:
import polars as pl

In [2]:
import os 
os.environ['POLARS_FMT_STR_LEN'] = str(50) 

In [202]:
df = pl.read_csv('../data/constoso_sales.csv', try_parse_dates=True)
df = df.with_columns(
    (pl.col('Quantity') * pl.col('Net Price')).round(2).alias('Sales Amount')
)
df.head()

Order Number,Line Number,Order Date,Delivery Date,Customer Name,Customer Gender,Customer Country,Customer Age,Store Name,Product Name,Color,Brand,Category,Subcategory,Quantity,Unit Price,Net Price,Unit Cost,Currency Code,Exchange Rate,Sales Amount
i64,i64,date,date,str,str,str,i64,str,str,str,str,str,str,i64,f64,f64,f64,str,f64,f64
284806,1,2017-10-18,2017-10-20,"""Eric Kennedy""","""Male""","""United States""",47,"""Online store""","""Contoso 512MB MP3 Player E51 Silver""","""Silver""","""Contoso""","""Audio""","""MP4&MP3""",7,11.691,10.288,5.958,"""USD""",1.0,72.02
285506,1,2017-10-25,2017-10-26,"""George Tooth""","""Male""","""Australia""",30,"""Online store""","""Contoso 512MB MP3 Player E51 Silver""","""Silver""","""Contoso""","""Audio""","""MP4&MP3""",1,11.691,11.691,5.958,"""AUD""",1.2967,11.69
311002,2,2018-07-07,2018-07-12,"""Caleb Greene""","""Male""","""Australia""",59,"""Online store""","""Contoso 512MB MP3 Player E51 Silver""","""Silver""","""Contoso""","""Audio""","""MP4&MP3""",6,12.99,12.99,6.62,"""AUD""",1.3484,77.94
366307,2,2020-01-11,2020-01-11,"""Isaac Siddins""","""Male""","""Australia""",25,"""Contoso Store Western Australia""","""Contoso 512MB MP3 Player E51 Blue""","""Blue""","""Contoso""","""Audio""","""MP4&MP3""",4,12.99,12.99,6.62,"""AUD""",1.4545,51.96
325708,3,2018-12-01,2018-12-02,"""Mike McQueen""","""Male""","""United States""",56,"""Online store""","""Contoso 512MB MP3 Player E51 Blue""","""Blue""","""Contoso""","""Audio""","""MP4&MP3""",2,12.99,11.5611,6.62,"""USD""",1.0,23.12


In [106]:
sales_by_cat = df.select(
    'Category',
    'Subcategory',
    pl.col('Sales Amount').sum().over('Category').alias('Sales Amt per Cat')
)
sales_by_cat.head()    

Category,Subcategory,Sales Amt per Cat
str,str,f64
"""Audio""","""MP4&MP3""",238356.0
"""Audio""","""MP4&MP3""",238356.0
"""Audio""","""MP4&MP3""",238356.0
"""Audio""","""MP4&MP3""",238356.0
"""Audio""","""MP4&MP3""",238356.0


In [107]:
sales_by_cat.filter(pl.col('Category')=='Audio').unique().head()

Category,Subcategory,Sales Amt per Cat
str,str,f64
"""Audio""","""MP4&MP3""",238356.0
"""Audio""","""Bluetooth Headphones""",238356.0
"""Audio""","""Recording Pen""",238356.0


In [108]:
df.shape, sales_by_cat.shape 

((13915, 21), (13915, 3))

In [109]:
(
    df
    .select(
        'Category',
        'Brand',
        'Subcategory',
        pl.col('Sales Amount').mean().over('Category', 'Brand').alias('Avg Sales per Cat and Brand')
    )
    .filter(
        (pl.col('Category')=='Computers') 
    )
    .unique()
    .sort('Brand')
    .head(10)
)

Category,Brand,Subcategory,Avg Sales per Cat and Brand
str,str,str,f64
"""Computers""","""Adventure Works""","""Laptops""",1797.371846
"""Computers""","""Adventure Works""","""Desktops""",1797.371846
"""Computers""","""Adventure Works""","""Monitors""",1797.371846
"""Computers""","""Contoso""","""Computers Accessories""",689.986652
"""Computers""","""Contoso""","""Projectors & Screens""",689.986652
"""Computers""","""Fabrikam""","""Laptops""",1982.066063
"""Computers""","""Proseware""","""Monitors""",1095.305012
"""Computers""","""Proseware""","""Laptops""",1095.305012
"""Computers""","""Proseware""","""Projectors & Screens""",1095.305012


In [110]:
from datetime import date

curr_yr = date.today().year
cust_birth_yr = curr_yr - pl.col('Customer Age')

(
    df
    .select(
        'Category',
        'Brand',
        'Customer Age',
        pl.col('Sales Amount').mean().over('Category', cust_birth_yr).alias('Avg Sales per Cat') 
    )
    .filter(pl.col('Category')=='Computers')
    .unique()
    .sort('Customer Age')
    .head(10)
)

Category,Brand,Customer Age,Avg Sales per Cat
str,str,i64,f64
"""Computers""","""Proseware""",19,1665.993509
"""Computers""","""Southridge Video""",19,1665.993509
"""Computers""","""Wide World Importers""",19,1665.993509
"""Computers""","""Contoso""",19,1665.993509
"""Computers""","""Fabrikam""",19,1665.993509
"""Computers""","""Adventure Works""",19,1665.993509
"""Computers""","""Fabrikam""",20,2094.541563
"""Computers""","""Adventure Works""",20,2094.541563
"""Computers""","""Wide World Importers""",20,2094.541563


In [111]:
(
    df
    .group_by('Category')
    .agg(pl.col('Sales Amount').max().alias('Max Sales Amt'))
    .with_columns(
        pl.col('Max Sales Amt').rank(descending=True).alias('Rank')
    )
    .sort('Rank')
)

Category,Max Sales Amt,Rank
str,f64,f64
"""TV and Video""",28999.9,1.0
"""Home Appliances""",28479.91,2.0
"""Computers""",19992.0,3.0
"""Cameras and camcorders """,10810.8,4.0
"""Cell phones""",5183.2,5.0
"""Music, Movies and Audio Books""",3041.88,6.0
"""Audio""",2871.2,7.0
"""Games and Toys""",2813.16,8.0


In [116]:
(
    df
    .group_by('Category', 'Subcategory')
    .agg(pl.col('Sales Amount').max().round().cast(pl.Int64).alias('Max Sales Amt'))
    .with_columns(
        pl.col('Max Sales Amt').rank(descending=True).over('Category').cast(pl.Int64).alias('Rank')
    )
    .filter(pl.col('Category').is_in(['Audio', 'Computers']))
    .sort(['Category', 'Rank'])
)

Category,Subcategory,Max Sales Amt,Rank
str,str,i64,i64
"""Audio""","""Recording Pen""",2871,1
"""Audio""","""Bluetooth Headphones""",2250,2
"""Audio""","""MP4&MP3""",2095,3
"""Computers""","""Projectors & Screens""",19992,1
"""Computers""","""Laptops""",19485,2
"""Computers""","""Desktops""",14535,3
"""Computers""","""Monitors""",11425,4
"""Computers""","""Printers, Scanners & Fax""",2508,5
"""Computers""","""Computers Accessories""",2424,6


### There is more...

In [239]:
max_sales_rank = (
    df
    .group_by('Category', 'Subcategory')
    .agg(pl.col('Sales Amount').max().round().cast(pl.Int64).alias('Max Sales Amt'))
    .with_columns(
        pl.col('Max Sales Amt').rank(descending=True).over('Category').cast(pl.Int64).alias('Rank')
    )
    .filter(pl.col('Category').is_in(['Audio', 'Computers']))
    .sort(['Category', 'Rank'])
)

In [242]:
max_sales_rank.with_columns(
    pl.col('Subcategory')
    .sort_by('Max Sales Amt')
    .head(3)
    .over('Category', mapping_strategy='join')
    .alias('Lowest 3 Subcat per Cat')
)

Category,Subcategory,Max Sales Amt,Rank,Lowest 3 Subcat per Cat
str,str,i64,i64,list[str]
"""Audio""","""Recording Pen""",2871,1,"[""MP4&MP3"", ""Bluetooth Headphones"", ""Recording Pen""]"
"""Audio""","""Bluetooth Headphones""",2250,2,"[""MP4&MP3"", ""Bluetooth Headphones"", ""Recording Pen""]"
"""Audio""","""MP4&MP3""",2095,3,"[""MP4&MP3"", ""Bluetooth Headphones"", ""Recording Pen""]"
"""Computers""","""Projectors & Screens""",19992,1,"[""Computers Accessories"", ""Printers, Scanners & Fax"", ""Monitors""]"
"""Computers""","""Laptops""",19485,2,"[""Computers Accessories"", ""Printers, Scanners & Fax"", ""Monitors""]"
"""Computers""","""Desktops""",14535,3,"[""Computers Accessories"", ""Printers, Scanners & Fax"", ""Monitors""]"
"""Computers""","""Monitors""",11425,4,"[""Computers Accessories"", ""Printers, Scanners & Fax"", ""Monitors""]"
"""Computers""","""Printers, Scanners & Fax""",2508,5,"[""Computers Accessories"", ""Printers, Scanners & Fax"", ""Monitors""]"
"""Computers""","""Computers Accessories""",2424,6,"[""Computers Accessories"", ""Printers, Scanners & Fax"", ""Monitors""]"


In [258]:
max_sales_rank.with_columns(
    pl.col('Subcategory')
    .sort_by('Max Sales Amt')
    .over('Category', mapping_strategy='explode')
    .alias('Subcategory Sorted by Max Sales Amt Ascending')
)

Category,Subcategory,Max Sales Amt,Rank,Subcategory Sorted by Max Sales Amt Ascending
str,str,i64,i64,str
"""Audio""","""Recording Pen""",2871,1,"""MP4&MP3"""
"""Audio""","""Bluetooth Headphones""",2250,2,"""Bluetooth Headphones"""
"""Audio""","""MP4&MP3""",2095,3,"""Recording Pen"""
"""Computers""","""Projectors & Screens""",19992,1,"""Computers Accessories"""
"""Computers""","""Laptops""",19485,2,"""Printers, Scanners & Fax"""
"""Computers""","""Desktops""",14535,3,"""Monitors"""
"""Computers""","""Monitors""",11425,4,"""Desktops"""
"""Computers""","""Printers, Scanners & Fax""",2508,5,"""Laptops"""
"""Computers""","""Computers Accessories""",2424,6,"""Projectors & Screens"""


In [257]:
(
    max_sales_rank
    .sort('Subcategory')
    .with_columns(
        pl.col('Subcategory')
        .sort_by('Max Sales Amt')
        .over('Category', mapping_strategy='explode')
        .alias('Subcategory Sorted by Max Sales Amt Ascending')
    )
)

Category,Subcategory,Max Sales Amt,Rank,Subcategory Sorted by Max Sales Amt Ascending
str,str,i64,i64,str
"""Audio""","""Bluetooth Headphones""",2250,2,"""MP4&MP3"""
"""Computers""","""Computers Accessories""",2424,6,"""Bluetooth Headphones"""
"""Computers""","""Desktops""",14535,3,"""Recording Pen"""
"""Computers""","""Laptops""",19485,2,"""Computers Accessories"""
"""Audio""","""MP4&MP3""",2095,3,"""Printers, Scanners & Fax"""
"""Computers""","""Monitors""",11425,4,"""Monitors"""
"""Computers""","""Printers, Scanners & Fax""",2508,5,"""Desktops"""
"""Computers""","""Projectors & Screens""",19992,1,"""Laptops"""
"""Audio""","""Recording Pen""",2871,1,"""Projectors & Screens"""
